## Import all the Libraries

In [2]:
import pandas as pd
import numpy as np
!pip install BeautifulSoup4
!pip install lxml
!pip install html5lib
print('Everything Imported')

Everything Imported


### Use Pandas to import table from Wikipedia page

In [3]:
from pandas.io.html import read_html
wikipage = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikitable = read_html(wikipage,index_col = 0, attrs={'class':'wikitable'})
print(len(wikitable[0]))

288


In [4]:
table = wikitable[0]
table.head()

,Borough,Neighbourhood
Postcode,,
M1A,Not assigned,Not assigned
M2A,Not assigned,Not assigned
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront


In [5]:
table.reset_index(inplace = True)
table.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Drop all the Rows where Borough Name is not assigned

In [6]:
table.drop(table[table.Borough == 'Not assigned'].index, inplace = True)
table.reset_index(drop = True, inplace = True)
table.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


### Rename all the unnamed Neighbourhoods to their respective Borough names

In [7]:
for a in table['Neighbourhood'].index:
    if table.iloc[a,2] == 'Not assigned':
        table.iloc[a,2] = table.iloc[a,1]

In [8]:
table.dropna(how = 'all', axis = 0, inplace = True)
table.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


### Merge the Neighbourhoods based on their Postcodes

In [9]:
toronto = table.groupby(['Postcode','Borough']).Neighbourhood.agg([', '.join])

In [11]:
toronto.reset_index(inplace = True)
toronto.rename(columns = {'join' : 'Neighbourhood'})

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


### Shape of the DataFrame

In [13]:
toronto.shape

(103, 3)